<h1 align=center> Sparse, Dense and Hybrid Search </h1>

In here we explore how to perform hybrid search using pinecone. Hybrid search is a technique that allows us to use both dense and sparse search to retrieve information from the vector database. Sparse search provides key word matching capabilities while dense search allows us to perform semantic searches (based on word meanings). Effectively combining these two may result in improved performance benefits.

We follow along with this guide [here](https://www.pinecone.io/learn/hybrid-search-intro/)

In [ ]:
# !pip install --upgrade datasets

### Prepare the Dataset

In [3]:
from datasets import load_dataset  # !pip install datasets

In [4]:
pubmed = load_dataset(
   'pubmed_qa',
   'pqa_labeled',
   split='train'
)
pubmed

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
})

In [8]:
contexts = []
# loop through the context passages
for record in pubmed['context']:
   # join context passages for each question and append to contexts list
   contexts.append('\n'.join(record['contexts']))
# view some of the contexts
for context in contexts[:2]:
   print(f"{context[:300]}...")

Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cel...
Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differenc...


### Creating Sparse Vectors
We will use the `BERT` tokenizer as a preliminary step.

### Note:
> Pinecone expects to receive sparse vectors in dictionary format. For example, the vector:
[0, 2, 9, 2, 5, 5]
> Would become:
{ "0": 1, "2": 2, "5": 2, "9": 1 }

> Each token is represented by a single key in the dictionary, and its frequency is counted by the respective key-value

In [5]:
from transformers import BertTokenizerFast  # !pip install transformers

In [6]:
# load bert tokenizer from huggingface
tokenizer = BertTokenizerFast.from_pretrained(
   'bert-base-uncased'
)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
# tokenize the context passage
inputs = tokenizer(
   contexts[0], padding=True, truncation=True,
   max_length=512
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

The article further states:
> We can reformat all of this logic into two functions; build_dict to transform input IDs into dictionaries and generate_sparse_vectors to handle the tokenization and dictionary creation.

In [28]:
from collections import Counter

In [66]:
def build_dict(input_batch):
 # store a batch of sparse embeddings
   sparse_emb = []
   # iterate through input batch
   for token_ids in input_batch:
       indices = []
       values = []
       # convert the input_ids list to a dictionary of key to frequency values
       d = dict(Counter(token_ids))
       for idx in d:
            indices.append(idx)
            values.append(float(d[idx]))                        # Extremely important to cast values as float
                                                                # Otherwise you get: SparseValuesMissingKeysError: Missing required keys in data in column `sparse_values`.
       sparse_emb.append({'indices': indices, 'values': values})
   # return sparse_emb list
   return sparse_emb


def generate_sparse_vectors(context_batch):
  # create batch of input_ids
  inputs = tokenizer(
          context_batch, padding=True,
          truncation=True,
          max_length=512,
          # special_tokens=False
  )['input_ids']
  # create sparse dictionaries
  sparse_embeds = build_dict(inputs)
  return sparse_embeds

### Dense Vectors

In [ ]:
# !pip install sentence-transformers

In [14]:
from sentence_transformers import SentenceTransformer

In [15]:
# load a sentence transformer model from huggingface
model = SentenceTransformer(
   'multi-qa-MiniLM-L6-cos-v1'
)

emb = model.encode(contexts[0])
emb.shape

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

(384,)

### Creating the Sparse-Dense Index
> The process for creating and using a sparse-dense index is almost identical to creating a pure dense index, the only chance being that upserts and queries must include an additional sparse_values parameter.


In [ ]:
!pip install pinecone-client==3.0.0

In [22]:
from pinecone import Pinecone, PodSpec  # !pip install pinecone-client

In [18]:
pinecone_key = '3c9fd500-de4d-40ed-8c77-12302b71e8ce'


Also
> To use a sparse-dense enabled index we must set `pod_type` to `s1` or `p1`, and `metric` to use `dotproduct`.

In [24]:
pc = Pinecone(api_key=pinecone_key)

# choose a name for your index
index_name = "hybrid-search-intro"

# create the index
pc.create_index(
   name = index_name,
   dimension = 384,  # dimensionality of dense model
   spec=PodSpec(environment="gcp-starter"),
   metric = "dotproduct",
)

#### Note:
The following assumes that no other index exists in the case of a starter plan on pinecone. If one does exist it raises an error, so the existing index has to be deleted first (manually or programatically) before creating a new one.

We could have opted to use whatever index is found to exist, but there is the problem of it being meant for a specific vector dimension that doesn't match ours.

### Populating Index

In [67]:
from tqdm.auto import tqdm

batch_size = 32
index = pc.Index(name=index_name)

for i in tqdm(range(0, len(contexts), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(contexts))
    # extract batch
    context_batch = contexts[i:i_end]
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]
    # add context passages as metadata
    meta = [{'context': context} for context in context_batch]
    # create dense vectors
    dense_embeds = model.encode(context_batch).tolist()
    # create sparse vectors
    sparse_embeds = generate_sparse_vectors(context_batch)

    vectors = []
    # loop through the data and create dictionaries for upserts
    for _id, sparse, dense, metadata in zip(
        ids, sparse_embeds, dense_embeds, meta
    ):

        vectors.append({
            'id': _id,
            'sparse_values': sparse,
            'values': dense,
            'metadata': metadata
        })

    # upload the documents to the new hybrid index
    index.upsert(vectors)


  0%|          | 0/32 [00:00<?, ?it/s]

In [69]:
# show index description after uploading the documents
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.01,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

### Querying
> Queries remain very similar to pure dense vector queries, with the exception being that we must include a sparse vector version of our query — alongside the typical dense vector representation.

In [77]:
def hybrid_scale(dense, sparse, alpha: float):
    # check alpha value is in range
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    # scale sparse and dense vectors to create hybrid search vecs
    hsparse = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    hdense = [v * alpha for v in dense]
    return hdense, hsparse


def hybrid_query(question, top_k, alpha):
   # convert the question into a sparse vector
   sparse_vec = generate_sparse_vectors([question])[0]
   # convert the question into a dense vector
   dense_vec = model.encode([question]).tolist()
   # scale alpha with hybrid_scale
   dense_vec, sparse_vec = hybrid_scale(
      dense_vec, sparse_vec, alpha
   )
   # query pinecone with the query parameters
   result = index.query(
      top_k=top_k,
      vector=dense_vec,
      sparse_vector=sparse_vec,
      include_metadata=True
   )
   # return search results as json
   return result

In [78]:
question = "Can clinicians use the PHQ-9 to assess depression in people with vision loss?"
hybrid_query(question, top_k=3, alpha=1)

{'matches': [{'id': '305',
              'metadata': {'context': 'The gap between evidence-based '
                                      'treatments and routine care has been '
                                      'well established. Findings from the '
                                      'Sequenced Treatments Alternatives to '
                                      'Relieve Depression (STAR*D) emphasized '
                                      'the importance of measurement-based '
                                      'care for the treatment of depression as '
                                      'a key ingredient for achieving response '
                                      'and remission; yet measurement-based '
                                      'care approaches are not commonly used '
                                      'in clinical practice.\n'
                                      'The Nine-Item Patient Health '
                                      'Questionnaire (PHQ-